In [2]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth
import csv

In [10]:
# Read the CSV file with census tract data
df = pd.read_excel("SDOH_2020_ZIPCODE_1_0.xlsx", sheet_name='Data')

In [13]:
df.head(10)

,YEAR,STATEFIPS,ZIPCODE,ZCTA,STATE,REGION,TERRITORY,POINT_ZIP,ACS_TOT_POP_WT_ZC,ACS_TOT_POP_US_ABOVE1_ZC,...,CEN_POPDENSITY_ZC,HIFLD_DIST_UC_ZP,POS_DIST_ED_ZP,POS_DIST_MEDSURG_ICU_ZP,POS_DIST_TRAUMA_ZP,POS_DIST_PED_ICU_ZP,POS_DIST_OBSTETRICS_ZP,POS_DIST_CLINIC_ZP,POS_DIST_ALC_ZP,CEN_AIAN_NH_IND
0,2020,15,96759,NaN,Hawaii,NaN,0,1,NaN,NaN,...,NaN,7.41,4.61,4.61,10.44,16.30,10.44,3.98,10.44,NaN
1,2020,2,99675,NaN,Alaska,NaN,0,1,NaN,NaN,...,NaN,205.60,209.98,210.08,211.62,210.08,209.98,39.64,210.73,NaN
2,2020,41,97010,NaN,Oregon,NaN,0,0,NaN,NaN,...,NaN,12.98,16.76,16.76,25.34,29.23,16.76,12.49,24.05,NaN
3,2020,6,93928,NaN,California,NaN,0,1,NaN,NaN,...,NaN,26.02,19.44,19.44,56.02,96.97,19.44,19.17,53.74,NaN
4,2020,6,95836,NaN,California,NaN,0,0,NaN,NaN,...,NaN,6.58,10.30,10.30,12.87,12.32,12.32,9.29,10.30,NaN
5,2020,15,96784,NaN,Hawaii,NaN,0,1,NaN,NaN,...,NaN,8.66,12.50,4.53,74.77,95.91,4.53,5.21,101.29,NaN
6,2020,2,99666,NaN,Alaska,NaN,0,1,NaN,NaN,...,NaN,492.72,133.29,134.02,158.41,495.26,133.29,101.53,133.29,NaN
7,2020,2,99764,NaN,Alaska,NaN,0,1,NaN,NaN,...,NaN,169.32,154.24,169.77,169.77,266.38,169.77,40.69,252.36,NaN
8,2020,2,99841,NaN,Alaska,NaN,0,1,NaN,NaN,...,NaN,55.93,51.93,51.93,51.93,NaN,51.93,49.20,51.93,NaN
9,2020,32,89033,NaN,Nevada,NaN,0,0,NaN,NaN,...,NaN,0.74,0.54,4.05,6.27,6.27,5.92,1.23,7.44,NaN


In [16]:
# Filter rows for Massachusetts – adjust the column name and value as needed
df_ma = df[df['STATE'] == "Massachusetts"]
df_ma.head(10)

,YEAR,STATEFIPS,ZIPCODE,ZCTA,STATE,REGION,TERRITORY,POINT_ZIP,ACS_TOT_POP_WT_ZC,ACS_TOT_POP_US_ABOVE1_ZC,...,CEN_POPDENSITY_ZC,HIFLD_DIST_UC_ZP,POS_DIST_ED_ZP,POS_DIST_MEDSURG_ICU_ZP,POS_DIST_TRAUMA_ZP,POS_DIST_PED_ICU_ZP,POS_DIST_OBSTETRICS_ZP,POS_DIST_CLINIC_ZP,POS_DIST_ALC_ZP,CEN_AIAN_NH_IND
227,2020,25,1001,1001.0,Massachusetts,Northeast,0,0,16064.0,15854.0,...,1403.92,5.59,3.38,3.38,3.52,3.52,3.38,2.76,3.38,0.0
228,2020,25,1059,1002.0,Massachusetts,Northeast,0,1,30099.0,29954.0,...,546.85,9.25,9.24,9.24,21.06,21.06,9.24,2.86,12.85,0.0
229,2020,25,1002,1002.0,Massachusetts,Northeast,0,0,30099.0,29954.0,...,546.85,7.81,7.78,7.78,18.13,18.13,7.78,0.40,13.41,0.0
230,2020,25,1004,1002.0,Massachusetts,Northeast,0,1,30099.0,29954.0,...,546.85,7.07,7.05,7.05,17.89,17.89,7.05,0.51,13.03,0.0
231,2020,25,1003,1003.0,Massachusetts,Northeast,0,0,11588.0,11588.0,...,16290.28,7.63,7.61,7.61,18.96,18.96,7.61,0.91,14.07,0.0
232,2020,25,1005,1005.0,Massachusetts,Northeast,0,0,5166.0,5145.0,...,116.77,18.74,12.75,12.75,18.74,32.37,13.00,12.29,8.62,0.0
233,2020,25,1007,1007.0,Massachusetts,Northeast,0,0,15080.0,14972.0,...,286.46,11.03,8.31,8.31,14.74,14.74,8.78,8.83,8.31,0.0
234,2020,25,1008,1008.0,Massachusetts,Northeast,0,0,1116.0,1111.0,...,20.75,18.52,11.23,11.23,18.71,18.71,16.81,5.36,16.81,0.0
235,2020,25,1009,1009.0,Massachusetts,Northeast,0,0,649.0,649.0,...,814.00,10.60,2.70,2.70,14.71,14.71,6.07,13.07,2.70,0.0
236,2020,25,1010,1010.0,Massachusetts,Northeast,0,0,3663.0,3643.0,...,105.43,16.05,7.60,7.60,20.74,20.74,8.51,9.05,7.60,0.0


In [17]:
df_ma.to_csv("sdoh_ma.csv", index=False)

In [3]:
driver = GraphDatabase.driver(
  "bolt://3.238.143.211:7687",
  auth=basic_auth("neo4j", "checker-shift-chips"))

In [4]:
sdoh_csv_data = []
with open("sdoh_ma.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sdoh_csv_data.append(row)

In [5]:
def upload_sdoh(tx, records):
    cypher_query = """
    UNWIND $records as record
    MERGE(zc:Zipcode {ZIPCODE: record.ZIPCODE})
    SET zc += record
    """
    tx.run(cypher_query, records=records)

with driver.session(database="neo4j") as session:
    session.execute_write(upload_sdoh, sdoh_csv_data)

In [6]:
with driver.session(database="neo4j") as session:
    session.execute_write(lambda tx: tx.run("""
        MATCH (p:Patient), (z:Zipcode)
        WHERE p.ZIP = "0"+ z.ZIPCODE
        MERGE (p)-[:LIVES_IN]->(z)
    """))

In [7]:
with driver.session(database="neo4j") as session:
    session.execute_write(lambda tx: tx.run("""
        MATCH (o:Organization), (z:Zipcode)
        WHERE o.ZIP = "0"+ z.ZIPCODE
        MERGE (o)-[:LOCATED_IN]->(z)
    """))

In [8]:
with driver.session(database="neo4j") as session:
    session.execute_write(lambda tx: tx.run("""
        MATCH (p:Provider), (z:Zipcode)
        WHERE p.ZIP = "0"+ z.ZIPCODE
        MERGE (p)-[:LOCATED_IN]->(z)
    """))